In [1]:
# Step to ensure that the venv is being used for the project not local copies, should point at .venv in project.
import sys, shutil
print("python:", sys.executable)
print("uv:", shutil.which("uv")) 

python: c:\Users\AndrzejPytel\source\Hackathon-2025-AP-Fork\.venv\Scripts\python.exe
uv: c:\Users\AndrzejPytel\source\Hackathon-2025-AP-Fork\.venv\Scripts\uv.EXE


In [2]:
# installs into the current Jupyter kernel environment
%pip install -U uv 
#! to run shell commands
!uv pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Using Python 3.13.7 environment at: c:\Users\AndrzejPytel\source\Hackathon-2025-AP-Fork\.venv
Audited 13 packages in 38ms


In [3]:
# LangChain + MCP Setup for Cinema Booking (HTTP-based for Jupyter)
import os
from dotenv import load_dotenv
import asyncio
import json
import requests
from typing import Dict, Any, List

# LangChain imports
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# Official MCP adapter imports for HTTP transport
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

# Load environment variables
load_dotenv()

print("✅ Updated imports with official MCP adapter loaded successfully!")

✅ Updated imports with official MCP adapter loaded successfully!


In [4]:
# MCP Client Setup using Official Adapter with HTTP Transport
import subprocess
import time

# Global MCP client for HTTP
mcp_client = None

async def create_mcp_tools():
    """Create MCP tools using the official LangChain MCP adapter with HTTP transport"""
    global mcp_client
    
    try:
        # Create MultiServerMCPClient with streamable_http transport for cinema
        mcp_client = MultiServerMCPClient({
            "cinema": {
                "transport": "streamable_http",
                "url": "http://localhost:8010"  # Cinema MCP server on port 8010
            }
        })
        
        # Get tools from the MCP server
        tools = await mcp_client.get_tools()
        print(f"Loaded {len(tools)} MCP tools: {[tool.name for tool in tools]}")
        return tools
        
    except Exception as e:
        print(f"Error connecting to Cinema MCP HTTP server: {e}")
        print("Make sure the cinema MCP server is running on port 8010")
        print("Run: cd src/mcp/cinema-mcp && uv run mcp dev main.py")
        return []

print("🔗 Cinema MCP HTTP adapter setup ready!")

🔗 Cinema MCP HTTP adapter setup ready!


In [5]:
# Load MCP Tools using Official Adapter
# The tools will be loaded dynamically when setting up the agent
# No need to manually create tool wrappers - the adapter handles this automatically
print("🛠️ Ready to load Cinema MCP tools via official adapter!")

🛠️ Ready to load Cinema MCP tools via official adapter!


In [6]:
async def setup_agent():
    """Setup LangChain agent with Cinema MCP tools using official adapter"""
    
    # Initialize LLM for Azure OpenAI
    # can get this from Azure Open AI service -> Azure AI Foundry Portal
    from langchain_openai import AzureChatOpenAI
    
    llm = AzureChatOpenAI(
        deployment_name=os.getenv("DEPLOYMENT_NAME"),  # Your Azure deployment name
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_version=os.getenv("AZURE_API_VERSION"), 
        temperature=1
    )
    
    # Load MCP tools using official adapter
    tools = await create_mcp_tools()
    
    if not tools:
        print("No MCP tools loaded. Make sure the Cinema MCP server is accessible.")
        return None
    
    print(f"Loaded {len(tools)} MCP tools: {[tool.name for tool in tools]}")
    
    # Create system prompt for cinema assistant
    system_prompt = """You are a helpful cinema assistant that can help users discover movies and make reservations.
    
    You have access to cinema MCP tools for movie showings, including:
    - Getting current movies playing with showtimes and availability
    - Searching for movies by title, genre, date, room, and seat availability
    - Getting detailed information about specific movie showings
    - Making movie reservations for customers
    - Viewing customer reservations
    - Canceling reservations
    
    When helping users:
    1. Use get_current_movies or search_movies to show available options
    2. Use get_movie_details with exact title, date, time, and room for specific showings
    3. For reservations, always collect: customer name, email, number of seats wanted
    4. Use make_reservation with the exact movie details from search results
    5. Be helpful and provide clear information about showtimes, pricing, and availability
    
    Always be friendly and guide users through the movie booking process step by step."""
    
    # Create prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # Create agent
    agent = create_tool_calling_agent(llm, tools, prompt)

    # memory
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    # Create agent executor with tool logging callback and verbose output
    agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)
    
    return agent_executor

# Initialize the agent (now async)
agent_executor = None
print("🤖 Cinema agent setup function ready! Run the next cell to initialize.")

🤖 Cinema agent setup function ready! Run the next cell to initialize.


In [7]:
# Initialize the agent with Cinema MCP tools
async def initialize_agent():
    """Initialize the agent with Cinema MCP tools"""
    global agent_executor
    print("Initializing cinema agent with MCP tools...")
    agent_executor = await setup_agent()
    if agent_executor:
        print("LangChain cinema agent with MCP tools ready!")
    else:
        print("Failed to initialize agent. Check Cinema MCP server connection.")

# Run the initialization
await initialize_agent()

Initializing cinema agent with MCP tools...
Error connecting to Cinema MCP HTTP server: unhandled errors in a TaskGroup (1 sub-exception)
Make sure the cinema MCP server is running on port 8010
Run: cd src/mcp/cinema-mcp && uv run mcp dev main.py
No MCP tools loaded. Make sure the Cinema MCP server is accessible.
Failed to initialize agent. Check Cinema MCP server connection.
Error connecting to Cinema MCP HTTP server: unhandled errors in a TaskGroup (1 sub-exception)
Make sure the cinema MCP server is running on port 8010
Run: cd src/mcp/cinema-mcp && uv run mcp dev main.py
No MCP tools loaded. Make sure the Cinema MCP server is accessible.
Failed to initialize agent. Check Cinema MCP server connection.


In [8]:
# User Input Handler + logged agent steps
async def process_user_input(user_input: str) -> str:
    """Process user input and return LLM response using Cinema MCP tools"""
    if not agent_executor:
        return "Agent not initialized. Please run the initialization cell first."
    
    try:
        # Use the agent to process the input and get intermediate steps
        result = await agent_executor.ainvoke({"input": user_input})
        output = result.get("output") or result.get("final_output") or ""

        # Print intermediate steps if present
        steps = result.get("intermediate_steps") or []
        for step in steps:
            action = None
            observation = None
            if isinstance(step, tuple) and len(step) == 2:
                action, observation = step
            elif isinstance(step, dict) and "action" in step:
                action = step.get("action")
                observation = step.get("observation")
            else:
                continue

            tool_name = getattr(action, "tool", getattr(action, "tool_name", "unknown"))
            tool_args = getattr(action, "tool_input", getattr(action, "input", None))
            print(f"\n--- Tool: {tool_name}")
            print(f"args: {tool_args}")
            if observation is not None:
                print(f"result: {observation}")
            print("---\n")

        return output
    except Exception as e:
        return f"Error processing request: {str(e)}"

# Interactive function for easy testing
async def ask_assistant(question: str):
    """Easy-to-use function for asking the cinema assistant"""
    print(f"🎬 User: {question}")
    print("🤖 Assistant:")
    
    response = await process_user_input(question)
    print(response)
    return response

print("💬 Cinema user input handler ready!")

💬 Cinema user input handler ready!


In [9]:
# Test Cinema MCP server connectivity and tools
async def test_mcp_connection():
    """Test Cinema MCP server connection and list available tools"""
    tools = await create_mcp_tools()
    if tools:
        print(f"Cinema MCP HTTP server connected successfully!")
        print(f"Available tools: {[tool.name for tool in tools]}")
        for tool in tools:
            print(f"  - {tool.name}: {tool.description}")
    else:
        print("Failed to connect to Cinema MCP HTTP server")
        print("Make sure to start the cinema MCP server first:")
        print("cd src/mcp/cinema-mcp && uv run mcp dev main.py")

# Test Cinema MCP HTTP connection
await test_mcp_connection()

Error connecting to Cinema MCP HTTP server: unhandled errors in a TaskGroup (1 sub-exception)
Make sure the cinema MCP server is running on port 8010
Run: cd src/mcp/cinema-mcp && uv run mcp dev main.py
Failed to connect to Cinema MCP HTTP server
Make sure to start the cinema MCP server first:
cd src/mcp/cinema-mcp && uv run mcp dev main.py


In [10]:
# 🚀 EXAMPLE USAGE - Run this cell after setting up your API key!

# Simple question about current movies
await ask_assistant("What movies are currently playing?")
print("\n" + "="*50 + "\n")

# Example 2: Search for specific type of movie
# await ask_assistant("I'm looking for action movies playing tomorrow. What do you have?")
# print("\n" + "="*50 + "\n")

🎬 User: What movies are currently playing?
🤖 Assistant:
Agent not initialized. Please run the initialization cell first.




In [ ]:
# Interactive chat loop — keep asking questions until you exit

# Try these example questions:
# - "What movies are playing today?"
# - "Show me action movies"
# - "I want to see Avatar tomorrow evening"
# - "Get me details for Avatar on 2025-09-25 at 19:30 in theater_a"
# - "Book 2 seats for Avatar on 2025-09-25 at 19:30 in theater_a for John Doe, john@email.com"
# - "Show my reservations for john@email.com"

async def chat_loop():
    print("🎬 Welcome to MovieMagic Cinema Assistant!")
    print("I can help you find movies, check showtimes, and make reservations.")
    print("Type 'exit' to quit. Press Enter on an empty line to skip.\n")
    
    while True:
        try:
            question = input("You: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nExiting.")
            break
        if not question:
            continue
        if question.lower() in ("exit", "quit", "q"):
            print("🎬 Thanks for using MovieMagic Cinema! Goodbye!")
            break
        await ask_assistant(question)

# Start the cinema chat loop
await chat_loop()

🎬 Welcome to MovieMagic Cinema Assistant!
I can help you find movies, check showtimes, and make reservations.
Type 'exit' to quit. Press Enter on an empty line to skip.



In [ ]:
# Cleanup function for HTTP MCP client
async def cleanup_mcp():
    """Cleanup MCP client and server resources"""
    global mcp_client
    if mcp_client:
        try:
            await mcp_client.close()
            print("Cinema MCP client closed")
        except Exception as e:
            print(f"Warning: Error closing Cinema MCP client: {e}")
        mcp_client = None

print("🧹 Cleanup function ready!")